In [1]:
import requests
import pandas as pd
import urllib
import json

In [2]:
bearer_token = pd.read_csv('bearer_token.txt', header = 0)

In [ ]:
bearer_token

In [ ]:
bearer_token['bearer_token'].iloc[0]

In [5]:
header = {'Authorization' : 'Bearer {}'.format(bearer_token['bearer_token'].iloc[0])}

In [6]:
bearer_token.keys()

Index(['bearer_token'], dtype='object')

In [7]:
endpoint_url = 'https://api.twitter.com/2/tweets/search/recent'

In [8]:
query = urllib.parse.quote('(#TedLasso OR Ted Lasso lang:en)')

In [9]:
query

'%28%23TedLasso%20OR%20Ted%20Lasso%20lang%3Aen%29'

In [10]:
my_api_url = endpoint_url + '?query={}'.format(query)

In [11]:
my_api_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%23TedLasso%20OR%20Ted%20Lasso%20lang%3Aen%29'

In [12]:
tweet_fields = 'public_metrics,created_at,author_id,lang'

In [13]:
my_api_url = endpoint_url + '?query={}&tweet.fields={}'.format(query, tweet_fields)

In [14]:
my_api_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%23TedLasso%20OR%20Ted%20Lasso%20lang%3Aen%29&tweet.fields=public_metrics,created_at,author_id,lang'

In [15]:
expansions = 'author_id'

In [16]:
url = endpoint_url + '?query={}&max_results=10&tweet.fields{}'.format(query, tweet_fields)

In [17]:
response = requests.request("GET", url, headers = header)

In [27]:
url_expansions = endpoint_url + '?query={}&max_results=100&tweet.fields={}&expansions={}&user.fields={}'.format(query, tweet_fields, expansions, 'username')

In [28]:
url_expansions

'https://api.twitter.com/2/tweets/search/recent?query=%28%23TedLasso%20OR%20Ted%20Lasso%20lang%3Aen%29&max_results=100&tweet.fields=public_metrics,created_at,author_id,lang&expansions=author_id&user.fields=username'

In [29]:
first_response = requests.request("GET", url_expansions, headers = header)

In [ ]:
first_response.text

In [31]:
first_dict = json.loads(first_response.text)

In [32]:
first_dict.keys()

dict_keys(['data', 'includes', 'meta'])

In [33]:
my_df = pd.DataFrame(first_dict['data'])

In [34]:
my_df

,id,author_id,text,lang,created_at,public_metrics
0,1467931711526817794,1444048793603907586,@franklinleonard Nick Mohammed absolutely snub...,en,2021-12-06T18:59:27.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
1,1467931702907621384,105834172,If this doesn’t look like a scene out of @TedL...,en,2021-12-06T18:59:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
2,1467931587136397316,1253066793809256448,Favorite Ted Lasso character? (Not Ted) Reply ...,en,2021-12-06T18:58:57.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
3,1467931103138828293,200617165,RT @Darveyinbed: Waking up with Hannah singing...,en,2021-12-06T18:57:02.000Z,"{'retweet_count': 98, 'reply_count': 0, 'like_..."
4,1467931007026286592,1004773208942206976,*Ted lasso opening credits starts playing* htt...,en,2021-12-06T18:56:39.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
...,...,...,...,...,...,...
95,1467921788390805506,1456361220701835268,#TedLasso https://t.co/usIXotMhHz,und,2021-12-06T18:20:01.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
96,1467921473025179653,28177237,"RT @TedLassoStache: And, hey, remember to focu...",en,2021-12-06T18:18:46.000Z,"{'retweet_count': 18, 'reply_count': 0, 'like_..."
97,1467921215151034375,17444763,RT @biscuitsoclock: rebecca be like 👼🏻 #tedlas...,en,2021-12-06T18:17:45.000Z,"{'retweet_count': 10, 'reply_count': 0, 'like_..."
98,1467921188651470852,526804114,RT @SudeikisNews: 🏆 | jason sudeikis and ted l...,en,2021-12-06T18:17:38.000Z,"{'retweet_count': 19, 'reply_count': 0, 'like_..."


Here is my data frame for tweets that mention Ted Lasso or #TedLasso. Having this information will allow me to filter through these tweet between negative and positive tweets. Then eventually i can compare the two. the question i am asking is if tweets have any effect for the winner of the best comedy series for the Emmy's. If there are more positive tweets than negative then it could help the winner of that category.

My next steps are to only have the text in the tweets and have a different data frame with that information. From there i can parse through the text and separate the text into a positive or negative category.

# From here and below is my new code for the analysis portion of the project

In [50]:
first_dict['meta']

{'newest_id': '1467931711526817794',
 'oldest_id': '1467921166463639562',
 'result_count': 100,
 'next_token': 'b26v89c19zqg8o3fpdy8isea290cgg9jcwsmwe17v3q7x'}

In [52]:
my_api_url2 = my_api_url + '&next_token={}&max_results=100'.format(first_dict['meta']['next_token'])

Here i am getting the next token and adding it my first response so i can get more that 100 tweets

In [53]:
second_response = requests.request("GET", my_api_url2, headers = header)

In [ ]:
second_response.text

In [56]:
my_df = my_df.append(pd.DataFrame(json.loads(second_response.text)['data']), ignore_index= True)

Here i am overriding my dataframe so i have all 200 tweets in one dataframe

In [57]:
my_df

,id,author_id,text,lang,created_at,public_metrics
0,1467931711526817794,1444048793603907586,@franklinleonard Nick Mohammed absolutely snub...,en,2021-12-06T18:59:27.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
1,1467931702907621384,105834172,If this doesn’t look like a scene out of @TedL...,en,2021-12-06T18:59:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
2,1467931587136397316,1253066793809256448,Favorite Ted Lasso character? (Not Ted) Reply ...,en,2021-12-06T18:58:57.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
3,1467931103138828293,200617165,RT @Darveyinbed: Waking up with Hannah singing...,en,2021-12-06T18:57:02.000Z,"{'retweet_count': 98, 'reply_count': 0, 'like_..."
4,1467931007026286592,1004773208942206976,*Ted lasso opening credits starts playing* htt...,en,2021-12-06T18:56:39.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
...,...,...,...,...,...,...
195,1467913966064132102,1102576999216959490,RT @oxentepipoca: Os indicados em Melhor Série...,en,2021-12-06T17:48:56.000Z,"{'retweet_count': 30, 'reply_count': 0, 'like_..."
196,1467913823810031616,1021986544436105217,RT @fay_maillardet: What Mar said👇🏻 If you're ...,en,2021-12-06T17:48:22.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c..."
197,1467913668042055682,2778398008,"@NYCFC That felt like watching Ted Lasso, but ...",en,2021-12-06T17:47:45.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
198,1467913606054354945,1244810432327512064,idk if Ted Lasso will win again since The Grea...,en,2021-12-06T17:47:30.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."


In [59]:
#second_dict = json.loads(second_response.text)

In [60]:
#second_dict['meta']

{'newest_id': '1467921129511731207',
 'oldest_id': '1467913504791441420',
 'result_count': 100,
 'next_token': 'b26v89c19zqg8o3fpdy8isdzcvq7xxcxoac5pf1ypuq2l'}

In [69]:
#my_api_url3 = my_api_url2 + '&next_token={}&max_results=100'.format(second_dict['meta']['next_token'])

In [70]:
#my_api_url3

'https://api.twitter.com/2/tweets/search/recent?query=%28%23TedLasso%20OR%20Ted%20Lasso%20lang%3Aen%29&tweet.fields=public_metrics,created_at,author_id,lang&next_token=b26v89c19zqg8o3fpdy8isea290cgg9jcwsmwe17v3q7x&max_results=100&next_token=b26v89c19zqg8o3fpdy8isdzcvq7xxcxoac5pf1ypuq2l&max_results=99'

In [71]:
#third_response = requests.request("GET", my_api_url3, headers = header)

In [72]:
#third_response.text

'{"errors":[{"parameters":{"max_results":["100","99"]},"message":"Duplicate parameters are not allowed: the `max_results` query parameter"},{"parameters":{"next_token":["b26v89c19zqg8o3fpdy8isea290cgg9jcwsmwe17v3q7x","b26v89c19zqg8o3fpdy8isdzcvq7xxcxoac5pf1ypuq2l"]},"message":"Duplicate parameters are not allowed: the `next_token` query parameter"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}'

Here I tried to add the next 100 tweets but ran into this error. I understand why I got the error but I don't know how to fix it.

In [73]:
my_df.to_csv('tedLassoAnalysis.csv')

Now I am converting my dataframe into a csv file so i can parse through the tweets.

In [74]:
lasso = pd.read_csv('tedLassoAnalysis.csv', encoding = 'utf-8')

In [75]:
lasso.keys()

Index(['Unnamed: 0', 'id', 'author_id', 'text', 'lang', 'created_at',
       'public_metrics'],
      dtype='object')

In [76]:
lasso.head()

,Unnamed: 0,id,author_id,text,lang,created_at,public_metrics
0,0,1467931711526817794,1444048793603907586,@franklinleonard Nick Mohammed absolutely snub...,en,2021-12-06T18:59:27.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
1,1,1467931702907621384,105834172,If this doesn’t look like a scene out of @TedL...,en,2021-12-06T18:59:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
2,2,1467931587136397316,1253066793809256448,Favorite Ted Lasso character? (Not Ted) Reply ...,en,2021-12-06T18:58:57.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
3,3,1467931103138828293,200617165,RT @Darveyinbed: Waking up with Hannah singing...,en,2021-12-06T18:57:02.000Z,"{'retweet_count': 98, 'reply_count': 0, 'like_..."
4,4,1467931007026286592,1004773208942206976,*Ted lasso opening credits starts playing* htt...,en,2021-12-06T18:56:39.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."


In [77]:
lasso['lang'].astype('category')

0      en
1      en
2      en
3      en
4      en
       ..
195    en
196    en
197    en
198    en
199    en
Name: lang, Length: 200, dtype: category
Categories (6, object): ['en', 'es', 'it', 'pt', 'tl', 'und']

In [78]:
lasso_posts = lasso[(lasso['lang']) == 'en']

In [79]:
lasso_posts['lang'].astype('category')

0      en
1      en
2      en
3      en
4      en
       ..
195    en
196    en
197    en
198    en
199    en
Name: lang, Length: 191, dtype: category
Categories (1, object): ['en']

Now I am retrieving all the tweets in English

In [80]:
lasso_posts['text']

0      @franklinleonard Nick Mohammed absolutely snub...
1      If this doesn’t look like a scene out of @TedL...
2      Favorite Ted Lasso character? (Not Ted) Reply ...
3      RT @Darveyinbed: Waking up with Hannah singing...
4      *Ted lasso opening credits starts playing* htt...
                             ...                        
195    RT @oxentepipoca: Os indicados em Melhor Série...
196    RT @fay_maillardet: What Mar said👇🏻 If you're ...
197    @NYCFC That felt like watching Ted Lasso, but ...
198    idk if Ted Lasso will win again since The Grea...
199    RT @BatatinhaGeek: As indicadas a 'Melhor Séri...
Name: text, Length: 191, dtype: object

In [81]:
lasso_posts['text'].iloc[0]

'@franklinleonard Nick Mohammed absolutely snubbed, his arc in season 2 ted lasso was incredible'

In [83]:
lasso_posts['text'].iloc[0].lower()

'@franklinleonard nick mohammed absolutely snubbed, his arc in season 2 ted lasso was incredible'

Making the tweet lowercase so its easier to work with

In [84]:
lowercase = [x.lower() for x in lasso_posts['text']]

In [85]:
lowercase[0]

'@franklinleonard nick mohammed absolutely snubbed, his arc in season 2 ted lasso was incredible'

In [86]:
import nltk
from nltk import tokenize as tk

In [87]:
tk.word_tokenize(lowercase[0])

['@',
 'franklinleonard',
 'nick',
 'mohammed',
 'absolutely',
 'snubbed',
 ',',
 'his',
 'arc',
 'in',
 'season',
 '2',
 'ted',
 'lasso',
 'was',
 'incredible']

In [91]:
nltk.download('opinion_lexicon')

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\erica\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\opinion_lexicon.zip.


True

In [92]:
from nltk.corpus import opinion_lexicon

Through some research i found this in the nltk.corpus library(?). It has words that are negative and also positive. (I'm proud that I found this :) )

In [96]:
opinion_lexicon.negative()[0:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [97]:
opinion_lexicon.positive()[0:10]

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation']

At this point i don't know how to go on. I have my library of negative and positive words but I want to find those negative/positive words in each tweet. I think this will help me with my hypothesis and separate the tweets into different dataframes(?) and then turn those into graphs which I can then compare. I think this is a step in the right direction and I am confident I will be able to support my hypothesis.